---

In [ ]:
#!pip install --user python-swiftclient python-keystoneclient --upgrade
#!apt-get install swiftclient
#!pip install geopandas

#Solve geopandas rtee problemm for sjoin
#!apt-get install -y libspatialindex-dev
#!pip install rtree

#!pip install nbdev

# If the graph do not display, try :
#!jupyter labextension install jupyterlab-plotly
# You may have to upgrade Node and Jupyter

# OCO2 - Display map and capture zone around the peak
Project for **Data For Good**, season 7. 

---

In [ ]:
# default_exp oco2mapfolium

In [ ]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
#%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Introduction


In [ ]:
# export
import pandas as pd
import geopandas as gpd
import numpy as np
from numpy import exp, loadtxt, pi, sqrt, log
import math
# import matplotlib
# import matplotlib.pyplot as plt
import swiftclient
import json
from io import StringIO
import folium
from folium import plugins
import geopy
from shapely.geometry import Polygon
from shapely.wkt import loads
# from geopy.distance import VincentyDistance # Removed geopy.distance.vincenty, use geopy.distance.geodesic instead. : https://geopy.readthedocs.io/en/stable/changelog_2xx.html#breaking-changes
# from geopy.distance import geodesic

In [ ]:
config_path = "../configs/config.json"

with open(config_path) as json_data_file:
    config = json.load(json_data_file)
    
def swift_con(config):
    user=config['swift_storage']['user']
    key=config['swift_storage']['key']
    auth_url=config['swift_storage']['auth_url']
    tenant_name=config['swift_storage']['tenant_name']
    auth_version=config['swift_storage']['auth_version']
    options = config['swift_storage']['options']
    return swiftclient.Connection(user=user,
                                  key=key,
                                  authurl=auth_url,
                                  os_options=options,
                                  tenant_name=tenant_name,
                                  auth_version=auth_version)

conn = swift_con(config)

## Retrieve Data

### Gaussian peaks with captured inventory

The CSV contains the gaussian peaks unfiltered, spatially joined with inventory data. The CSV is stored on the GitHub.

In [ ]:
path_peaks = "../dataset/peaks_and_invent/peaks_and_invent_1410.csv"
df = pd.read_csv(path_peaks, sep=",")
df = gpd.GeoDataFrame(df)
df['geometry'] = df['geometry'].apply(loads)
df.crs = {'init': 'epsg:4326'}
df.head()

,sounding_id,latitude,longitude,orbit,slope,intercept,amplitude,sigma,delta,R,...,index_Iron and steel,number_Iron and steel,index_Combustion,number_Combustion,index_Non ferrous metals,number_Non ferrous metals,index_Mineral oil,number_Mineral oil,index_Coke ovens,number_Coke ovens
0,2014100205581236,39.072670,107.781944,1339,0.004097,396.359779,71.886003,27.375463,1.047594,0.524609,...,[nan],0,[nan],0,[nan],0,[nan],0,[nan],0
1,2014100205581377,39.148392,107.769295,1339,0.003033,396.212893,100.136482,16.587700,2.408331,0.653852,...,[nan],0,[nan],0,[nan],0,[nan],0,[nan],0
2,2014100205581436,39.192081,107.744568,1339,0.002226,396.212991,112.860315,11.120307,4.048877,0.788405,...,[nan],0,[nan],0,[nan],0,[nan],0,[nan],0
3,2014100205581505,39.235714,107.719788,1339,0.001005,396.234629,123.853792,9.757608,5.063794,0.934664,...,[nan],0,[nan],0,[nan],0,[nan],0,[nan],0
4,2014100205581575,39.275524,107.707375,1339,-0.000225,396.247235,125.116723,10.329770,4.832087,0.900564,...,[nan],0,[nan],0,[nan],0,[nan],0,[nan],0


### Inventory Data

The CSV is stored on the GitHub.

In [ ]:
year = "2016"

path_invent = "https://raw.githubusercontent.com/dataforgoodfr/batch7_satellite_ges/master/dataset/Output%20inventory%20data/Merge%20of%20peaks/CO2_emissions_peaks_merged_"+year+".csv"
invent = pd.read_csv(path_invent, sep=",", index_col=0)

invent = gpd.GeoDataFrame(invent, geometry=gpd.points_from_xy(invent.longitude, invent.latitude))
invent.crs = {'init': 'epsg:4326'}

invent = invent[invent['longitude'].notna()]
invent = invent[invent['latitude'].notna()]
invent = invent[invent['CO2/CO2e emissions (in tonnes per year)'].notna()]

invent_types = invent['CO2/CO2e emissions source'].unique()
print("Types of inventory: ")
print(invent_types)

invent.head()

Types of inventory: 
['City' 'Oil power plant' 'Coal power plant' 'Gas power plant'
 'Cement and Lime' 'Glass' 'Power and heat' 'Ceramics' 'Pulp and paper'
 'Chemicals' 'Iron and steel' 'Combustion' 'Non ferrous metals'
 'Mineral oil' 'Coke ovens']


,latitude,longitude,Data source,CO2/CO2e emissions source,CO2/CO2e emissions (in tonnes per year),CO2 or CO2e,geometry
0,43.653226,-79.383184,Opendatasoft,City,16151019.00,CO2,POINT (-79.3831843 43.653226)
4,45.802578,9.086356,Opendatasoft,City,3728678.00,CO2,POINT (9.086356 45.802578)
5,37.668900,-122.080800,Opendatasoft,City,861854.00,CO2e,POINT (-122.0808 37.6689)
6,35.689634,139.692101,Opendatasoft,City,27611000.00,CO2e,POINT (139.6921007 35.6896342)
11,-10.249091,-48.324286,Opendatasoft,City,589055.31,CO2e,POINT (-48.324286 -10.249091)


---

## Map Visualization & Statistics methods

### Inventory Map

To display only the inventory data on a folium map:

In [ ]:
d={}
for cl in invent_types:
    d["{0}".format(cl)]=cl
    
d['City']

'City'

In [ ]:
# export
def inventory_map_only(invent):
    """
    Create map with inventory only
    :param invent: GeoDataFrame, Dataframe containing all inventory.
    :return:
    """
    # Initialize Map
    inventory_map = folium.Map([43, 0], zoom_start=4, tiles=None)
    folium.TileLayer("CartoDB dark_matter", name="Dark mode").add_to(inventory_map)
    folium.TileLayer("OpenStreetMap", name="Open Street Map").add_to(inventory_map)

    d={}
    invent_types = invent['CO2/CO2e emissions source'].unique()
    for cl in invent_types:
        d["{0}".format(cl)]=folium.FeatureGroup(name=cl).add_to(inventory_map)
        
    for index, row in invent.iterrows():
        radius = 1
        color="#368534" # green

        tooltip =  "["+str(round(row['latitude'],2))+" ; "+str(round(row['longitude'],2))+"]"
        pop = str(round(row['CO2/CO2e emissions (in tonnes per year)'],0))
        title = "" + str(round(row['CO2/CO2e emissions (in tonnes per year)'],0)) + " T/y " + row['CO2 or CO2e']
        popup_html = """<h4>"""+title+"""</h4><p>"""+tooltip+"""</p>""" + """<p>"""+ str(row['CO2/CO2e emissions source']) + ", from " + str(row['Data source']) + """</p>"""
        popup=folium.Popup(popup_html, max_width=450)

        d[row['CO2/CO2e emissions source']].add_child(folium.CircleMarker(location=(row["latitude"],
                                      row["longitude"]),
                            radius=radius,
                            color=color,
                            popup=popup,
                            tooltip= str(row['CO2/CO2e emissions source']),
                            fill=True))

    folium.map.LayerControl(collapsed=False).add_to(inventory_map)

    plugins.Fullscreen(
        position='topright',
        title='Expand me',
        title_cancel='Exit me',
        force_separate_button=True
    ).add_to(inventory_map)
    
    print("Saving "+year+ " inventory map...")
    inventory_map.save("../front/invent_maps/inventory_map_"+year+".html")
    return inventory_map

### Draw source zone

In [ ]:
#export
def peaks_capture_map(peaks, invent, mapbox_token = None, month="1908"):
    """
    Create map with peaks (marker + capture zone) and inventory
    :param peaks: GeoDataFrame, Dataframe containing the peaks we want to display.
    :param plants: GeoDataFrame, Dataframe containing all registered plants.
    :param plants_coal: GeoDataFrame, Dataframe containing all registered coal plants.
    :param cities: GeoDataFrame, Dataframe containing all registered big cities.
    :return:
    """
    # Initialize Map
    peaks_capture = folium.Map([40, -100], zoom_start=3, tiles=None)
    folium.TileLayer("CartoDB dark_matter", name="Dark mode").add_to(peaks_capture)
    folium.TileLayer("OpenStreetMap", name="Open Street Map").add_to(peaks_capture)
    if mapbox_token is not None:
        folium.TileLayer(tiles="Mapbox", name="Satellite", API_key=mapbox_token).add_to(peaks_capture)
    
    
    # Adding detected peaks
    peaks_group = folium.FeatureGroup(name="Peaks").add_to(peaks_capture)
    peaks_group_capture = folium.FeatureGroup(name=" - 50km Capture Zone", show=True).add_to(peaks_capture)
    for index, row in peaks.iterrows():
        radius = row["amplitude"]/20
        tooltip =  "["+str(round(row['latitude'],2))+" ; "+str(round(row['longitude'],2))+"]"
        color="#FF3333" # red
        sounding = str(row['sounding_id'])
        date = str(row['date'])
        orbit = str(row['orbit'])  
        
        popup_html="""<h4>"""+tooltip+"""</h4>"""+date+"""<p>sounding_id: """+sounding+"""</br>orbit: """+orbit+"""</p>"""
        popup_html+='<p><input type="button" value="Show plot"'
        # Injecting JavaScript in popup to fire the Dash Callback
        popup_html+='onclick="\
            let bco_input = parent.document.getElementById(\'input_sounding\'); \
            let lastValue = bco_input.value;'
        popup_html+=f'bco_input.value = \'{sounding}\';'
        popup_html+="let bco_event = new Event('input', { bubbles: true });\
            bco_event.simulated = true;\
            let tracker = bco_input._valueTracker;\
            if (tracker) {\
            tracker.setValue(lastValue);\
            }\
            bco_input.dispatchEvent(bco_event);\
            elt.dispatchEvent(new Event('change'));\
            \"/></p>"
        
        popup=folium.Popup(popup_html, max_width=450)
        
        peaks_group_capture.add_child(folium.GeoJson(row['geometry'], name=" - Capture Zone", tooltip=sounding)) # , popup=popup
        
        peaks_group.add_child(folium.CircleMarker(location=(row["latitude"],
                                      row["longitude"]),
                            radius=radius,
                            color=color,
                            tooltip=sounding,
                            popup=popup,
                            fill=True))

    d={}
    invent_types = invent['CO2/CO2e emissions source'].unique()
    inventory = folium.FeatureGroup(name="Inventory").add_to(peaks_capture)
    for cl in invent_types:
        d["{0}".format(cl)]=folium.plugins.FeatureGroupSubGroup(inventory, name=" - "+cl).add_to(peaks_capture)
        
    for index, row in invent.iterrows():
        radius = 1
        color="#368534" # green

        tooltip =  "["+str(round(row['latitude'],2))+" ; "+str(round(row['longitude'],2))+"]"
        pop = str(round(row['CO2/CO2e emissions (in tonnes per year)'],0))
        title = "" + str(round(row['CO2/CO2e emissions (in tonnes per year)'],0)) + " T/y " + row['CO2 or CO2e']
        popup_html = """<h4>"""+title+"""</h4><p>"""+tooltip+"""</p>""" + """<p>"""+ str(row['CO2/CO2e emissions source']) + ", from " + str(row['Data source']) + """</p>"""
        popup=folium.Popup(popup_html, max_width=450)

        d[row['CO2/CO2e emissions source']].add_child(folium.CircleMarker(location=(row["latitude"],
                                      row["longitude"]),
                            radius=radius,
                            color=color,
                            tooltip= str(row['CO2/CO2e emissions source']),
                            popup=popup,
                            fill=True))

    peaks_capture.keep_in_front(peaks_group)
    
    folium.map.LayerControl(collapsed=False).add_to(peaks_capture)
    
    plugins.Fullscreen(
        position='topleft',
        title='Expand me',
        title_cancel='Exit me',
        force_separate_button=True
    ).add_to(peaks_capture)

    minimap = plugins.MiniMap(position='bottomleft')
    peaks_capture.add_child(minimap)
    
    # print("Saving "+month+ " peak map...")
    # peaks_capture.save("../front/peak_maps/peaks_capture_map_"+month+".html")
    return peaks_capture

## Examples and Sanity Checks

### Map Examples

In [ ]:
inventory_map_only(invent)

Saving 2016 inventory map...


FileNotFoundError: [Errno 2] No such file or directory: '../front/invent_maps/inventory_map_2016.html'

In [ ]:
# Read config file to get mapbox token
config_file = "../configs/config.json"
with open(config_file) as json_data_file:
    config = json.load(json_data_file)
mapbox_token = config['mapbox_token']


In [ ]:
peaks_capture_map(df[30:45], invent, mapbox_token)

In [ ]:
#export
def peaks_capture_map(peaks, invent, mapbox_token = None, month="1908"):
    """
    Create map with peaks (marker + capture zone) and inventory
    :param peaks: GeoDataFrame, Dataframe containing the peaks we want to display.
    :param plants: GeoDataFrame, Dataframe containing all registered plants.
    :param plants_coal: GeoDataFrame, Dataframe containing all registered coal plants.
    :param cities: GeoDataFrame, Dataframe containing all registered big cities.
    :return:
    """
    # Initialize Map
    peaks_capture = folium.Map([40, -100], zoom_start=3, tiles=None)
    folium.TileLayer("CartoDB dark_matter", name="Dark mode").add_to(peaks_capture)
    folium.TileLayer("OpenStreetMap", name="Open Street Map").add_to(peaks_capture)
    if mapbox_token is not None:
        folium.TileLayer(tiles="Mapbox", name="Satellite", API_key=mapbox_token).add_to(peaks_capture)
    
    
    # Adding detected peaks
    peaks_group = folium.FeatureGroup(name="Peaks").add_to(peaks_capture)
    peaks_group_capture = folium.FeatureGroup(name=" - 50km Capture Zone", show=True).add_to(peaks_capture)
    for index, row in peaks.iterrows():
        radius = row["amplitude"]/20
        tooltip =  "["+str(round(row['latitude'],2))+" ; "+str(round(row['longitude'],2))+"]"
        color="#FF3333" # red
        sounding = str(row['sounding_id'])
        date = str(row['date'])
        orbit = str(row['orbit'])  
        
        popup_html="""<h4>"""+tooltip+"""</h4>"""+date+"""<p>sounding_id: """+sounding+"""</br>orbit: """+orbit+"""</p>"""
        popup_html+='<p><input type="button" value="Show plot"'
        # Injecting JavaScript in popup to fire the Dash Callback
        popup_html+='onclick="\
            let bco_input = parent.document.getElementById(\'input_sounding\'); \
            let lastValue = bco_input.value;'
        popup_html+=f'bco_input.value = \'{sounding}\';'
        popup_html+="let bco_event = new Event('input', { bubbles: true });\
            bco_event.simulated = true;\
            let tracker = bco_input._valueTracker;\
            if (tracker) {\
            tracker.setValue(lastValue);\
            }\
            bco_input.dispatchEvent(bco_event);\
            elt.dispatchEvent(new Event('change'));\
            \"/></p>"
        
        popup=folium.Popup(popup_html, max_width=450)
        
        peaks_group_capture.add_child(folium.GeoJson(row['geometry'], name=" - Capture Zone", tooltip=sounding)) # , popup=popup
        
        peaks_group.add_child(folium.CircleMarker(location=(row["latitude"],
                                      row["longitude"]),
                            radius=radius,
                            color=color,
                            tooltip=sounding,
                            popup=popup,
                            fill=True))
        
        folium.PolyLine([[row['latitude'],row['longitude']],[row['latitude']+row['windspeed_u'],row['longitude']+row['windspeed_v']]], color="green", weight=2.5, opacity=1).add_to(peaks_capture)
    

    peaks_capture.keep_in_front(peaks_group)
    
    
    folium.map.LayerControl(collapsed=False).add_to(peaks_capture)
    
    plugins.Fullscreen(
        position='topleft',
        title='Expand me',
        title_cancel='Exit me',
        force_separate_button=True
    ).add_to(peaks_capture)

    minimap = plugins.MiniMap(position='bottomleft')
    peaks_capture.add_child(minimap)
    
    # print("Saving "+month+ " peak map...")
    # peaks_capture.save("../front/peak_maps/peaks_capture_map_"+month+".html")
    return peaks_capture

In [ ]:
peaks_capture_map(df, invent, mapbox_token)

In [ ]:
df[30:35].sounding_id

In [ ]:
df[30:35].windspeed_u

In [ ]:
df[30:35].windspeed_v

---

In [ ]:
from nbdev.export import *
notebook2script()

Converted explorations.ipynb.
Converted index.ipynb.
Converted oco2peak-datasets-points-isoles.ipynb.
Converted oco2peak-datasets.ipynb.
Converted oco2peak-find_peak.ipynb.
Converted oco2peak-map.ipynb.
Converted oco2peak-mapfolium.ipynb.
Converted oco2peak-nc4_convert.ipynb.
Converted oco2peak-swift_utils.ipynb.
Converted oco2peak_find_source.ipynb.


In [ ]:
folium.__version__

In [ ]:
folium._version